In [1]:
import pandas
import numpy
import pygsheets
import datetime
import pytz
import re

In [2]:
# define the excel file to save tables in
current_time = datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%Y-%m-%d_T%H%M%S")
excel_writer = pandas.ExcelWriter('Global-LNG-Terminals-SummarySheets-'+current_time+'.xlsx')

In [3]:
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
spreadsheet = gc.open_by_key('1tcS6Wd-Wp-LTDpLzFgJY_RSNDnbyubW3J_9HKIAys4A')
#spreadsheet = gc.open_by_key('1MrghwBeCz8Tzgua7CWGg_KoXKVZsV7r0kHMYHYqnNTg') # July 2022 terminals version
#spreadsheet = gc.open_by_key('1FVuw76YU-jmAzfSWJ2ixwBQB_wXEK9aGCa4Q6OCOUzY') # July 2022 with May 2023 updates for Asia gas tracker

terms_df_orig = spreadsheet.worksheet('title', 'Terminals').get_as_df(start='A3')

# drop any empty/imcomplete rows (those without a wiki page)
terms_df_orig = terms_df_orig.loc[terms_df_orig.Wiki != '']

#get other relevant sheets
owners_df_orig = spreadsheet.worksheet('title', 'Terminal operators/owners (1/3)').get_as_df(start='A2')
owners_df_orig.set_index('ComboID', inplace=True)

parent_metadata_df = spreadsheet.worksheet('title', 'Parent metadata (3/3)').get_as_df(start='A2')
parent_metadata_df.set_index('Parent', inplace=True)

In [4]:
terms_df_orig.replace('--', numpy.nan, inplace=True)

In [5]:
region_df_orig = spreadsheet.worksheet('title', 'Country dictionary').get_as_df(start='A2')
#region_df_agt = region_df_orig.loc[region_df_orig['AsiaGasTracker']=='Yes']

region_df_touse = region_df_orig.copy()
#region_df_agt.copy()

In [6]:
region_df_touse_cleaned = region_df_touse.loc[(region_df_touse.Region!='--')&
                                            (region_df_touse.SubRegion!='--')]
multiindex_region_subregion = region_df_touse_cleaned.groupby(['Region','SubRegion'])['Country'].count().index
multiindex_region_subregion

MultiIndex([(  'Africa',                 'Northern Africa'),
            (  'Africa',              'Sub-Saharan Africa'),
            ('Americas', 'Latin America and the Caribbean'),
            ('Americas',                'Northern America'),
            (    'Asia',                    'Central Asia'),
            (    'Asia',                    'Eastern Asia'),
            (    'Asia',              'South-eastern Asia'),
            (    'Asia',                   'Southern Asia'),
            (    'Asia',                    'Western Asia'),
            (  'Europe',                  'Eastern Europe'),
            (  'Europe',                 'Northern Europe'),
            (  'Europe',                 'Southern Europe'),
            (  'Europe',                  'Western Europe'),
            ( 'Oceania',       'Australia and New Zealand'),
            ( 'Oceania',                       'Melanesia'),
            ( 'Oceania',                      'Micronesia'),
            ( 'Oceania',

### create specific dataframes for region, country_ratios, owners_df

In [7]:
owners_df_touse = owners_df_orig.loc[owners_df_orig['Country'].isin(region_df_touse['Country'].tolist())]
terms_df_touse = terms_df_orig.loc[terms_df_orig['Country'].isin(region_df_touse['Country'].tolist())]

In [8]:
status_list = ['Proposed', 
               'Construction', 
               'Shelved', 
               'Cancelled', 
               'Operating', 
               'Idle', 
               'Mothballed', 
               'Retired']
country_list = sorted(set(region_df_touse['Country'].tolist()))
region_list = sorted(set(region_df_touse['Region'].tolist()))
subregion_list = sorted(set(region_df_touse['SubRegion'].tolist()))

In [9]:
excel_status_list = ['Proposed', 
                     'Construction', 
                     'In Development (Proposed + Construction)', 
                     'Shelved', 
                     'Cancelled', 
                     'Operating', 
                     'Idle', 
                     'Mothballed', 
                     'Retired']
excel_status_list_with_countries = ['Country']+excel_status_list

In [10]:
# regions - subregions
#############################
# Africa - Sub-Saharan Africa
# Africa - Northern Africa
# Americas - Latin America and the Caribbean
# Americas - North America
# Asia - Southern Asia
# Asia - Western Asia
# Asia - South-eastern Asia
# Asia - East Asia
# Asia - Central Asia
# Europe - Southern Europe
# Europe - Eastern Europe
# Europe - Western Europe
# Europe - Northern Europe
# Oceania	- Polynesia
# Oceania	- Australia and New Zealand
# Oceania	- Melanesia
# Oceania	- Micronesia

### no. export trains by country/region, project status

In [11]:
terms_df_subset = terms_df_touse.copy()[(terms_df_touse['FacilityType']=='Export') & (terms_df_touse['Fuel']=='LNG')]

ntrains_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
ntrains_by_region = pandas.DataFrame(0, columns=status_list, index=multiindex_region_subregion)

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    ntrains_by_country[status] = terms_df_subset_status.groupby('Country').size()

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    ntrains_by_region[status] = terms_df_subset_status.groupby(['Region','SubRegion']).size()

# fille NaN with 0.0
ntrains_by_region = ntrains_by_region.fillna(0)
ntrains_by_country = ntrains_by_country.fillna(0)

ntrains_by_region['In Development (Proposed + Construction)'] = ntrains_by_region[['Proposed','Construction']].sum(axis=1)
ntrains_by_country['In Development (Proposed + Construction)'] = ntrains_by_country[['Proposed','Construction']].sum(axis=1)

ntrains_by_country = ntrains_by_country[excel_status_list]
ntrains_by_region = ntrains_by_region[excel_status_list]

ntrains_by_region.index.names = ['Region','Subregion']
ntrains_by_country.index.name = 'Country'

ntrains_by_region.loc['Total',:] = ntrains_by_region.sum(axis=0).values
ntrains_by_country.loc['Total',:] = ntrains_by_country.sum(axis=0).values

ntrains_by_region = ntrains_by_region.loc[~(ntrains_by_region==0).all(axis=1)]

ntrains_by_region.replace(0,'',inplace=True)
ntrains_by_country.replace(0,'',inplace=True)

ntrains_by_region.to_excel(excel_writer, sheet_name='LNG export trains by region')#, index=False)
ntrains_by_country.to_excel(excel_writer, sheet_name='LNG export trains by country')#, index=False)

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


### no. import trains by country/region, project status

In [12]:
terms_df_subset = terms_df_touse.copy()[(terms_df_touse['FacilityType']=='Import') & (terms_df_touse['Fuel']=='LNG')]

ntrains_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
ntrains_by_region = pandas.DataFrame(0, columns=status_list, index=multiindex_region_subregion)

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    ntrains_by_country[status] = terms_df_subset_status.groupby('Country').size()

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    ntrains_by_region[status] = terms_df_subset_status.groupby(['Region','SubRegion']).size()

# fille NaN with 0.0
ntrains_by_region = ntrains_by_region.fillna(0)
ntrains_by_country = ntrains_by_country.fillna(0)

ntrains_by_region['In Development (Proposed + Construction)'] = ntrains_by_region[['Proposed','Construction']].sum(axis=1)
ntrains_by_country['In Development (Proposed + Construction)'] = ntrains_by_country[['Proposed','Construction']].sum(axis=1)

ntrains_by_country = ntrains_by_country[excel_status_list]
ntrains_by_region = ntrains_by_region[excel_status_list]

ntrains_by_region.index.names = ['Region','Subregion']
ntrains_by_country.index.name = 'Country'

ntrains_by_region.loc['Total',:] = ntrains_by_region.sum(axis=0).values
ntrains_by_country.loc['Total',:] = ntrains_by_country.sum(axis=0).values

ntrains_by_region = ntrains_by_region.loc[~(ntrains_by_region==0).all(axis=1)]

ntrains_by_region.replace(0,'',inplace=True)
ntrains_by_country.replace(0,'',inplace=True)

ntrains_by_region.to_excel(excel_writer, sheet_name='LNG import trains by region')#, index=False)
ntrains_by_country.to_excel(excel_writer, sheet_name='LNG import trains by country')#, index=False)

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


### capacity (mtpa) of import trains by country/region, project status

In [13]:
terms_df_subset = terms_df_touse.copy()[(terms_df_touse['FacilityType']=='Import') & (terms_df_touse['Fuel']=='LNG')]

cap_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
cap_by_region = pandas.DataFrame(0, columns=status_list, index=multiindex_region_subregion)

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    cap_by_country[status] = terms_df_subset_status.groupby('Country')['CapacityInMtpa'].sum()

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    cap_by_region[status] = terms_df_subset_status.groupby(['Region','SubRegion'])['CapacityInMtpa'].sum()

#fille NaN with 0.0
cap_by_region = cap_by_region.fillna(0)
cap_by_country = cap_by_country.fillna(0)

cap_by_region['In Development (Proposed + Construction)'] = cap_by_region[['Proposed','Construction']].sum(axis=1)
cap_by_country['In Development (Proposed + Construction)'] = cap_by_country[['Proposed','Construction']].sum(axis=1)

cap_by_country = cap_by_country[excel_status_list]
cap_by_region = cap_by_region[excel_status_list]

cap_by_region.index.names = ['Region','Subregion']
cap_by_country.index.name = 'Country'

cap_by_region.loc['Total',:] = cap_by_region.sum(axis=0).values
cap_by_country.loc['Total',:] = cap_by_country.sum(axis=0).values

cap_by_region = cap_by_region.loc[~(cap_by_region==0).all(axis=1)]

cap_by_region.replace(0,'',inplace=True)
cap_by_country.replace(0,'',inplace=True)

cap_by_region.to_excel(excel_writer, 'LNG import capacity by region')
cap_by_country.to_excel(excel_writer, 'LNG import capacity by country')

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


In [14]:
cap_by_region

Proposed Construction  \
Region   Subregion                                               
Africa   Northern Africa                                         
         Sub-Saharan Africa                   1.5          1.7   
Americas Latin America and the Caribbean    40.88        16.06   
         Northern America                    0.05                
Asia     Eastern Asia                      195.15        113.2   
         South-eastern Asia                  38.0          8.6   
         Southern Asia                      67.36         30.0   
         Western Asia                                      2.0   
Europe   Eastern Europe                      4.49         1.54   
         Northern Europe                    34.37          3.8   
         Southern Europe                    45.95         4.04   
         Western Europe                     66.64        19.24   
Oceania  Australia and New Zealand            8.3         2.33   
Total                                      502.69       202.51   

                                         In Development (Proposed + Construction)  \
Region   Subregion                                                                  
Africa   Northern Africa                                                            
         Sub-Saharan Africa                                                   3.2   
Americas Latin America and the Caribbean                                    56.94   
         Northern America                                                    0.05   
Asia     Eastern Asia                                                      308.35   
         South-eastern Asia                                                  46.6   
         Southern Asia                                                      97.36   
         Western Asia                                                         2.0   
Europe   Eastern Europe                                                      6.03   
         Northern Europe                                                    38.17   
         Southern Europe                                                    49.99   
         Western Europe                                                     85.88   
Oceania  Australia and New Zealand                                          10.63   
Total                                                                       705.2   

                                         Shelved  Cancelled Operating  Idle  \
Region   Subregion                                                            
Africa   Northern Africa                    6.15      10.64             5.7   
         Sub-Saharan Africa                 0.88       7.70       2.5         
Americas Latin America and the Caribbean   16.28      24.97     77.14         
         Northern America                            259.52      59.8  25.0   
Asia     Eastern Asia                      40.26      52.17    532.35         
         South-eastern Asia                 27.9      35.17     58.15         
         Southern Asia                     25.54      72.45      65.5         
         Western Asia                       11.1       8.00      72.8  22.9   
Europe   Eastern Europe                     9.31       7.30      4.58         
         Northern Europe                              27.16     43.78         
         Southern Europe                    2.65      47.68     79.91         
         Western Europe                               12.40      72.0         
Oceania  Australia and New Zealand           1.0       1.75                   
Total                                     141.07     566.91   1068.51  53.6   

                                         Mothballed Retired  
Region   Subregion                                           
Africa   Northern Africa                        4.2          
         Sub-Saharan Africa                                  
Americas Latin America and the Caribbean                     
         Northern America                      4

### capacity (mtpa) of export trains by country/region, project status

In [146]:
terms_df_subset = terms_df_touse.copy()[(terms_df_touse['FacilityType']=='Export') & (terms_df_touse['Fuel']=='LNG')]

cap_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
cap_by_region = pandas.DataFrame(0, columns=status_list, index=multiindex_region_subregion)

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    cap_by_country[status] = terms_df_subset_status.groupby('Country')['CapacityInMtpa'].sum()

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    cap_by_region[status] = terms_df_subset_status.groupby(['Region','SubRegion'])['CapacityInMtpa'].sum()

#fille NaN with 0.0
cap_by_region = cap_by_region.fillna(0)
cap_by_country = cap_by_country.fillna(0)

cap_by_region['In Development (Proposed + Construction)'] = cap_by_region[['Proposed','Construction']].sum(axis=1)
cap_by_country['In Development (Proposed + Construction)'] = cap_by_country[['Proposed','Construction']].sum(axis=1)

cap_by_country = cap_by_country[excel_status_list]
cap_by_region = cap_by_region[excel_status_list]

cap_by_region.index.names = ['Region','Subregion']
cap_by_country.index.name = 'Country'

cap_by_region.loc['Total',:] = cap_by_region.sum(axis=0).values
cap_by_country.loc['Total',:] = cap_by_country.sum(axis=0).values

cap_by_region = cap_by_region.loc[~(cap_by_region==0).all(axis=1)]

cap_by_region.replace(0,'',inplace=True)
cap_by_country.replace(0,'',inplace=True)

cap_by_region.to_excel(excel_writer, 'LNG export capacity by region')
cap_by_country.to_excel(excel_writer, 'LNG export capacity by country')

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


## owner analysis

In [147]:
# this cell takes the terminals data set (call it "terms_df_all", here we're using the indev terminals subset)
# and loops throw every row, breaks up the parent and % ownership strings,
# and outputs a dataframe owner_parent_calculations_df that has a bunch of different useful info related
# to parent shares, capacity, etc.
owner_parent_calculations_df = pandas.DataFrame() # empty df for storing owner/parent stuff

for idx,row in terms_df_touse.loc[terms_df_touse.Fuel=='LNG'].iterrows():
    parent_string = row.Parent #.values[0]
    if row.TerminalID=='T0292':
        print(parent_string)

    # the two lines below go through the parent entries in the sheet and split them,
    # recording the percents and parents in separate lists, to be dealt with later
    parent_list = re.sub(' \[.*?\]', '', parent_string).split('; ') # all entries must have an "Owner [%]" syntax, so [unknown %] is included
    percent_list = [float(i.rstrip('%'))/100. for i in re.findall('\\d+(?:\\.\\d+)?%', parent_string)]

    # if there are fewer recorded percents than number of parents
    if parent_list.__len__()!=percent_list.__len__():
        # if there are no percents, make them all equal
        
        if percent_list==[]:
            percent_list = [1/parent_list.__len__() for i in parent_list]
        # otherwise, get the difference and distribute the remainder equally among remaining parents
        else:
            #print(row.ComboID)
            nmissing = parent_list.__len__()-percent_list.__len__()
            # distribute nans evenly
            total = numpy.nansum(percent_list)
            leftover = 1-total
            percent_list += [leftover/nmissing]*nmissing
            #print(numpy.array(percent_list).sum())

    # now go through each parent and associated percent ownership
    # and store it alongside the project's name along with capacity, status, region, etc.
    # this is stored inside the big dataframe owner_parent_calculations_df
    for p_idx,parent in enumerate(parent_list):
        if row.TerminalID=='T0292':
            print(parent_list)
        # this if statement should not really be used
        if parent not in parent_metadata_df.index.to_list():
            print(parent, " doesn't exist as a parent")
            parent_metadata_df.loc[parent,'ParentHQCountry']='Unknown'
        owner_parent_calculations_df = pandas.concat([owner_parent_calculations_df, 
                                                      pandas.DataFrame([{'Parent':parent, 'ComboID':row.ComboID,
                                                                         'TerminalID':row.TerminalID,
                                                                         'FractionOwnership':percent_list[p_idx],
                                                                         'PercentString':'{:.2f}%'.format(percent_list[p_idx]*100),
                                                                         'ParentHQCountry':parent_metadata_df.loc[parent_metadata_df.index==parent]['ParentHQCountry'].values[0],
                                                                         'ParentHQRegion':parent_metadata_df.loc[parent_metadata_df.index==parent]['ParentHQRegion'].values[0],
                                                                         'TerminalCountry':row.Country,
                                                                         'TerminalRegion':row.Region,
                                                                         'Status':row.Status,
                                                                         'FacilityType':row['FacilityType'],
                                                                         'CapacityInMtpa':row.CapacityInMtpa,
                                                                         'ProjectWiki':row.Wiki,
                                                                         'ProjectName':row.TerminalName+' '+str(row.UnitName)}])])

owner_parent_calculations_df.reset_index(drop=True, inplace=True)
owner_parent_calculations_df['CapacityOwnedInMtpa'] = owner_parent_calculations_df.FractionOwnership*owner_parent_calculations_df.CapacityInMtpa
owner_parent_calculations_df['ParentCapacityString'] = owner_parent_calculations_df.Parent + ' (' + owner_parent_calculations_df.CapacityOwnedInMtpa.round(1).astype(str).replace('nan','unknown') + ' mtpa)'

Deutsche Regas  doesn't exist as a parent
台灣中油天然氣事業部  doesn't exist as a parent
Petronas [80.00%]; Government of Sarawak [10.00%]; Mitsubishi [10.00%]
['Petronas', 'Government of Sarawak', 'Mitsubishi']
['Petronas', 'Government of Sarawak', 'Mitsubishi']
['Petronas', 'Government of Sarawak', 'Mitsubishi']
Petronas [80.00%]; Government of Sarawak [10.00%]; Mitsubishi [10.00%]
['Petronas', 'Government of Sarawak', 'Mitsubishi']
['Petronas', 'Government of Sarawak', 'Mitsubishi']
['Petronas', 'Government of Sarawak', 'Mitsubishi']
Petronas [80.00%]; Government of Sarawak [10.00%]; Mitsubishi [10.00%]
['Petronas', 'Government of Sarawak', 'Mitsubishi']
['Petronas', 'Government of Sarawak', 'Mitsubishi']
['Petronas', 'Government of Sarawak', 'Mitsubishi']
江苏泓海能源有限公司  doesn't exist as a parent
中交营口液化天然气有限公司  doesn't exist as a parent
中海油绿能港浙江宁波能源有限公司  doesn't exist as a parent
Höegh LNG  doesn't exist as a parent


In [148]:
owner_parent_calculations_df.loc[owner_parent_calculations_df.TerminalID=='T0292'].groupby(
    ['Parent','ParentHQCountry','ComboID','Status']).agg({'Status':['count']}).unstack().droplevel(axis=1, level=[0,1])

Status                                         Idle
Parent                ParentHQCountry ComboID      
Government of Sarawak Malaysia        T029201     1
                                      T029202     1
                                      T029203     1
Mitsubishi            Japan           T029201     1
                                      T029202     1
                                      T029203     1
Petronas              Malaysia        T029201     1
                                      T029202     1
                                      T029203     1

### ntrains export by parent company, project status

In [149]:
# owners_ntrains_by_status_df = \
#     owner_parent_calculations_df.loc[owner_parent_calculations_df['FacilityType']=='Export'].groupby(
#     ['Parent','ParentHQCountry','Status']).agg({'Status':['count']}).unstack().droplevel(axis=1, level=[0,1])

owners_ntrains_by_status_df = \
    owner_parent_calculations_df.loc[owner_parent_calculations_df['FacilityType']=='Export'].groupby(
    ['Parent','ParentHQCountry','Status'])[['FractionOwnership']].sum().unstack().droplevel(axis=1, level=[0])
#.agg({'Status':['count']}).unstack().droplevel(axis=1, level=[0,1])

# owners_ntrains_by_status_df.loc['Total',:] = owners_ntrains_by_status_df.sum(axis=0, min_count=0).values
#owners_ntrains_by_status_df.loc['Total','Country'] = ''
owners_ntrains_by_status_df

,Status,Cancelled,Construction,Idle,Mothballed,Operating,Proposed,Retired,Shelved
Parent,ParentHQCountry,,,,,,,,
A-Property,Russia,NaN,NaN,NaN,NaN,NaN,0.45,NaN,NaN
ADNOC,United Arab Emirates,2.00,NaN,NaN,NaN,2.1,1.40,NaN,NaN
AECOM,United States,NaN,NaN,NaN,NaN,NaN,1.50,NaN,NaN
AES Corporation,United States,NaN,NaN,NaN,NaN,NaN,0.20,NaN,NaN
AfricaGlobal Schaffer,United States,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...
Yemen Gas Company,Yemen,NaN,NaN,NaN,0.334,NaN,NaN,NaN,NaN
Yemen’s General Authority for Social Security and Pensions,Yemen,NaN,NaN,NaN,0.100,NaN,NaN,NaN,NaN
Zachry Hastings,United States,NaN,NaN,NaN,NaN,NaN,0.20,NaN,NaN


In [150]:
# owners_ntrains_by_status_df = \
#     owner_parent_calculations_df.loc[owner_parent_calculations_df['FacilityType']=='Export'].groupby(
#     ['Parent','ParentHQCountry','ComboID','Status']).agg({'Status':['count']}).unstack().droplevel(axis=1, level=[0,1])
owners_ntrains_by_status_df = \
    owner_parent_calculations_df.loc[owner_parent_calculations_df['FacilityType']=='Export'].groupby(
    ['Parent','ParentHQCountry','Status'])[['FractionOwnership']].sum().unstack().droplevel(axis=1, level=[0])

owners_ntrains_by_status_df = owners_ntrains_by_status_df.reindex(columns=status_list)
owners_ntrains_by_status_df = owners_ntrains_by_status_df.reset_index().set_index('Parent')
owners_ntrains_by_status_df.columns.name = None

owners_ntrains_by_status_df['In Development (Proposed + Construction)'] = owners_ntrains_by_status_df[['Proposed','Construction']].sum(axis=1)

owners_ntrains_by_status_df = owners_ntrains_by_status_df.rename(columns={'Parent':'Parent Company',
                                                                          'ParentHQCountry':'Country'})
# rearrange the order of the columns for output
owners_ntrains_by_status_df = owners_ntrains_by_status_df[excel_status_list_with_countries]

# totals_row = owners_ntrains_by_status_df.sum(axis=0, min_count=0)
# totals_row.name = 'Total'
# owners_ntrains_by_status_df = owners_ntrains_by_status_df.append(totals_row)
owners_ntrains_by_status_df.loc['Total',:] = owners_ntrains_by_status_df.sum(axis=0, min_count=0).values
owners_ntrains_by_status_df.loc['Total','Country'] = ''

owners_ntrains_by_status_df = owners_ntrains_by_status_df.replace(numpy.nan, '')
owners_ntrains_by_status_df = owners_ntrains_by_status_df.replace(0, '')

owners_ntrains_by_status_df.to_excel(excel_writer, 'LNG export trains by owner')

In [151]:
owners_ntrains_by_status_df.tail()

,Country,Proposed,Construction,In Development (Proposed + Construction),Shelved,Cancelled,Operating,Idle,Mothballed,Retired
Parent,,,,,,,,,,
Yemen’s General Authority for Social Security and Pensions,Yemen,,,,,,,,0.1,
Zachry Hastings,United States,0.2,,0.2,,,,,,
Zhejiang Energy,China,0.1,,0.1,,,,,,
Électricité de France,France,,,,,0.25,,,,
Total,,156.0004,43.0004,199.0008,15.4,186.0004,162.9939,4.0,9.0,18.0


### ntrains import by parent company, project status

In [152]:
owners_ntrains_by_status_df = \
    owner_parent_calculations_df.loc[owner_parent_calculations_df['FacilityType']=='Import'].groupby(
    ['Parent','ParentHQCountry','Status'])[['FractionOwnership']].sum().unstack().droplevel(axis=1, level=[0])

owners_ntrains_by_status_df = owners_ntrains_by_status_df.reindex(columns=status_list)
owners_ntrains_by_status_df = owners_ntrains_by_status_df.reset_index().set_index('Parent')
owners_ntrains_by_status_df.columns.name = None

owners_ntrains_by_status_df['In Development (Proposed + Construction)'] = owners_ntrains_by_status_df[['Proposed','Construction']].sum(axis=1)

owners_ntrains_by_status_df = owners_ntrains_by_status_df.rename(columns={'Parent':'Parent Company',
                                                                          'ParentHQCountry':'Country'})
# rearrange the order of the columns for output
owners_ntrains_by_status_df = owners_ntrains_by_status_df[excel_status_list_with_countries]

# totals_row = owners_ntrains_by_status_df.sum(axis=0, min_count=0)
# totals_row.name = 'Total'
# owners_ntrains_by_status_df = owners_ntrains_by_status_df.append(totals_row)
owners_ntrains_by_status_df.loc['Total',:] = owners_ntrains_by_status_df.sum(axis=0, min_count=0).values
owners_ntrains_by_status_df.loc['Total','Country'] = ''

owners_ntrains_by_status_df = owners_ntrains_by_status_df.replace(numpy.nan, '')
owners_ntrains_by_status_df = owners_ntrains_by_status_df.replace(0, '')

owners_ntrains_by_status_df.to_excel(excel_writer, 'LNG import trains by owner')

### capacity export by parent company, project status

In [153]:
owners_capacity_by_status_df = \
    owner_parent_calculations_df.loc[owner_parent_calculations_df['FacilityType']=='Export'].groupby(
    ['Parent','ParentHQCountry','Status']).agg({'CapacityOwnedInMtpa':['sum']}).unstack().droplevel(axis=1, level=[0,1])

owners_capacity_by_status_df = owners_capacity_by_status_df.reindex(columns=status_list)
owners_capacity_by_status_df = owners_capacity_by_status_df.reset_index().set_index('Parent')
owners_capacity_by_status_df.columns.name = None

owners_capacity_by_status_df['In Development (Proposed + Construction)'] = owners_capacity_by_status_df[['Proposed','Construction']].sum(axis=1)

owners_capacity_by_status_df = owners_capacity_by_status_df.rename(columns={'Parent':'Parent Company',
                                                                          'ParentHQCountry':'Country'})
# rearrange the order of the columns for output
owners_capacity_by_status_df = owners_capacity_by_status_df[excel_status_list_with_countries]

# totals_row = owners_capacity_by_status_df.sum(axis=0, min_count=0)
# totals_row.name = 'Total'
# owners_capacity_by_status_df = owners_capacity_by_status_df.append(totals_row)
owners_capacity_by_status_df.loc['Total',:] = owners_capacity_by_status_df.sum(axis=0, min_count=0).values
owners_capacity_by_status_df.loc['Total','Country'] = ''

owners_capacity_by_status_df = owners_capacity_by_status_df.replace(numpy.nan, '')
owners_capacity_by_status_df = owners_capacity_by_status_df.replace(0, '')

owners_capacity_by_status_df.to_excel(excel_writer, 'LNG export capacity by owner')

### capacity import by parent company, project status

In [154]:
owners_capacity_by_status_df = \
    owner_parent_calculations_df.loc[owner_parent_calculations_df['FacilityType']=='Import'].groupby(
    ['Parent','ParentHQCountry','Status']).agg({'CapacityOwnedInMtpa':['sum']}).unstack().droplevel(axis=1, level=[0,1])

owners_capacity_by_status_df = owners_capacity_by_status_df.reindex(columns=status_list)
owners_capacity_by_status_df = owners_capacity_by_status_df.reset_index().set_index('Parent')
owners_capacity_by_status_df.columns.name = None

owners_capacity_by_status_df['In Development (Proposed + Construction)'] = owners_capacity_by_status_df[['Proposed','Construction']].sum(axis=1)

owners_capacity_by_status_df = owners_capacity_by_status_df.rename(columns={'Parent':'Parent Company',
                                                                          'ParentHQCountry':'Country'})
# rearrange the order of the columns for output
owners_capacity_by_status_df = owners_capacity_by_status_df[excel_status_list_with_countries]

# totals_row = owners_capacity_by_status_df.sum(axis=0, min_count=0)
# totals_row.name = 'Total'
# owners_capacity_by_status_df = owners_capacity_by_status_df.append(totals_row)
owners_capacity_by_status_df.loc['Total',:] = owners_capacity_by_status_df.sum(axis=0, min_count=0).values
owners_capacity_by_status_df.loc['Total','Country'] = ''

owners_capacity_by_status_df = owners_capacity_by_status_df.replace(numpy.nan, '')
owners_capacity_by_status_df = owners_capacity_by_status_df.replace(0, '')

owners_capacity_by_status_df.to_excel(excel_writer, 'LNG import capacity by owner')

### no. of operating terminals by start year, facility type (1980–2023)

In [155]:
year_indices = list(range(1980,2023))

terms_by_start_year_df = pandas.DataFrame(0.0, index=year_indices, columns=['Import terminals', 'Export terminals'])

terms_started_df = terms_df_touse[(terms_df_touse['Status'].isin(['Operating'])) &
                              (terms_df_touse['Fuel']=='LNG') & (terms_df_touse['FacilityType']=='Import')]
terms_by_start_year_df['Import terminals'] = terms_started_df.groupby('StartYearEarliest').size()

terms_started_df = terms_df_touse[(terms_df_touse['Status'].isin(['Operating'])) &
                              (terms_df_touse['Fuel']=='LNG') & (terms_df_touse['FacilityType']=='Export')]
terms_by_start_year_df['Export terminals'] = terms_started_df.groupby('StartYearEarliest').size()

terms_by_start_year_df.index.name = 'Start year'

terms_by_start_year_df.loc['Total',:] = terms_by_start_year_df.sum(axis=0).values

terms_by_start_year_df.replace(numpy.nan,'',inplace=True)

terms_by_start_year_df.to_excel(excel_writer, 'LNG terminals by start year')

### capacity of operating terminals by start year, facility type (1980–2021)

In [156]:
year_indices = list(range(1980,2023))

capacity_by_start_year_df = pandas.DataFrame(0.0, index=year_indices, columns=['Import terminals', 'Export terminals'])

terms_started_df = terms_df_touse.loc[(terms_df_touse['Status'].isin(['Operating'])) &
                                      (terms_df_touse['Fuel']=='LNG') & 
                                      (terms_df_touse['FacilityType']=='Import')]
capacity_by_start_year_df['Import terminals'] = terms_started_df.groupby('StartYearEarliest')['CapacityInMtpa'].sum()

terms_started_df = terms_df_touse.loc[(terms_df_touse['Status'].isin(['Operating'])) &
                                      (terms_df_touse['Fuel']=='LNG') & 
                                      (terms_df_touse['FacilityType']=='Export')]
capacity_by_start_year_df['Export terminals'] = terms_started_df.groupby('StartYearEarliest')['CapacityInMtpa'].sum(min_count=0)

capacity_by_start_year_df.loc['Total',:] = capacity_by_start_year_df.sum(axis=0).values

capacity_by_start_year_df.index.name = 'Start year'
capacity_by_start_year_df.replace(numpy.nan,'',inplace=True)

# totals_row = capacity_by_start_year_df.sum(axis=0)
# totals_row.name = 'Total'
# capacity_by_start_year_df = capacity_by_start_year_df.append(totals_row)

capacity_by_start_year_df.to_excel(excel_writer, 'LNG capacity by start year')

# cost info

# save excel file

In [157]:
excel_writer.close()

In [158]:
print(current_time)

2023-11-28_T144125
